# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [289]:
import pandas as pd
import numpy as np
import glob
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [517]:
#Test and vizualise the shape of the data
from IPython.display import display
guinea = DATA_FOLDER+'/ebola/guinea_data/*.csv'
liberia = DATA_FOLDER+'/ebola/liberia_data/*.csv'
sierra = DATA_FOLDER+'/ebola/sl_data/*.csv'


s = pd.read_csv(glob.glob(sierra)[0])
for file in glob.glob(sierra)[1:]:
    s = pd.concat([s,pd.read_csv(file)],axis = 0)
frames = [s,g,l]
s = s.rename(index=str,columns={'date':'Date','variable':'Variable'})

s.Date = pd.to_datetime(s.Date)




s['Country'] = 'Sierra-Leone'
#data = pd.merge(s,g, how='outer')
#data = pd.merge(data, l , how='outer')
#data = data.merge(s,how='outer')
#data.Date = data.Date.map(lambda x: str(x.year)+'-'+str(x.month))
#data = data.set_index(['Country','Date','Variable'])
#s.Date = s.Date.map(lambda x: str(x.year)+'-'+str(x.month))
#s = s.fillna(0)
s = s[s['Variable'].isin(['death_confirmed','new_confirmed'])]
#s[s.Variable.str.contains('death')].Bo.dropna().astype('int64').sum()
#s = s.fillna(0)
s.Date = s.Date.map(lambda x: '-'.join(str(x).split('-')[:2]))
#s.Variable.loc[s.Variable.str.contains('death')] = 'Death'
#s.Variable.loc[s.Variable.str.contains('new')] = 'New Cases'
s = s.set_index(['Country','Date','Variable'])['National']#.sum(axis=1)
s = s.dropna(axis=0, how = all)
s = pd.DataFrame(s)
s = s.astype('int64').groupby(level=2)
i = 0
for group in s:
    if i== 0:
        print(group[1].max())
    else:
        group[1].mean()
s
#s = s[s.values.]

#s.groupby(['Country','Date','Variable']).sum()
#data
#g.head()
#display(l.head())
#display(s.head())

National    1708
dtype: int64
National    111
dtype: int64


In [461]:
g = pd.read_csv(glob.glob(guinea)[0])
for file in glob.glob(guinea)[1:]:
    g = pd.concat([g,pd.read_csv(file)],axis = 0)
g = g.rename(columns={'Description':'Variable'})
g.Date = pd.to_datetime(g.Date)
g['Country'] = 'Guinea'
g = g[g['Variable'].isin(['New cases of confirmed','New deaths registered'])]
g = g.fillna(0)
g.Date = g.Date.map(lambda x: '-'.join(str(x).split('-')[:2]))
g = g.set_index(['Country','Date','Variable']).astype('int64')['Totals']
#g = g.set_index(['Country','Date','Variable']).astype('int64')
g = g.groupby(level=[0,1,2]).mean()
g = pd.DataFrame(g)
g.columns = ['Daily Average']
g

Daily Average
Country Date    Variable                             
Guinea  2014-08 New cases of confirmed        12.4000
                New deaths registered          3.7500
        2014-09 New cases of confirmed        13.0000
                New deaths registered          3.5625
        2014-10 New cases of confirmed         6.0000
                New deaths registered         15.0000

In [436]:
l = pd.read_csv(glob.glob(liberia)[0])
for file in glob.glob(liberia)[1:]:
    l = pd.concat([l,pd.read_csv(file)], axis = 0)
l.Date = pd.to_datetime(l.Date)
l['Country'] = 'Liberia'
#l = l[l['Variable'].isin(["Total death/s in confirmed, probable, suspected cases", "Total suspected cases",
#                         "Total probable cases", "Total confirmed cases"])]
l = l[l['Variable'].isin(["Total death/s in confirmed, probable, suspected cases"])]
l = l.fillna(0)
#l.Date = l.Date.map(lambda x: str(x.year)+'-'+str(x.month))
l.Date = l.Date.map(lambda x: '-'.join(str(x).split('-')[:2]))
l = l.set_index(['Country','Date','Variable']).astype('int64')
display(l)
l = l.sum(axis=1).groupby(level=[0,1,2]).max()
#l.Date = pd.to_datetime(l.index[1])
l = pd.DataFrame(l).sort_index(0,1, True)
display(l)
lastmonth_death = 0
for value in l.index:
    curr = l.loc[value].copy()
    new_val = curr- lastmonth_death
    lastmonth_death = curr 
    l.loc[value] = new_val
l

Bomi County  \
Country Date    Variable                                                          
Liberia 2014-06 Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
        2014-07 Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...            1   
                Total death/s in confirmed, probable, suspected...            1   
                Total death/s in confirmed, probable, suspected...            2   
                Total death/s in confirmed, probable, suspected...            2   
                Total death/s in confirmed, probable, suspected...            2   
                Total death/s in confirmed, probable, suspected...            2   
        2014-08 Total death/s in confirmed, probable, suspected...            8   
                Total death/s in confirmed, probable, suspected...           10   
                Total death/s in confirmed, probable, suspected...           13   
                Total death/s in confirmed, probable, suspected...           14   
                Total death/s in confirmed, probable, suspected...           14   
                Total death/s in confirmed, probable, suspected...           14   
                Total death/s in confirmed, probable, suspected...           14   
                Total death/s in confirmed, probable, suspected...           29   
                Total death/s in confirmed, probable, suspected...           33   
        2014-09 Total death/s in confirmed, probable, suspected...           34   
                Total death/s in confirmed, probable, suspected...           34   
                Total death/s in confirmed, probable, suspected...           34   
...                                                                         ...   
                Total death/s in confirmed, probable, suspected...           64   
                Total death/s in confirmed, probable, suspected...           66   
                Total death/s in confirmed, probable, suspected...           66   
                Total death/s in confirmed, probable, suspected...           66   
                Total death/s in confirmed, probable, suspected...           68   
        2014-10 Total death/s in confirmed, probable, suspected...           68   
                Total death/s in confirmed, probable, suspected...           68   
                Total death/s in confirmed, probable, suspected...           68   
                Total death/s in confirmed, probable, suspected...           68   
                Total death/s in confirmed, probable, suspected...           68   
                Total death/s in confirmed, probable, suspected...           69   
                Total death/s in confirmed, probable, suspected...           69   
                Total death/s in confirmed, probable, suspected...           69   
                Total death/s in confirmed, probable, suspected...           69   
                Total death/s in confirmed, probable, suspected...            0   
                Total death/s in confirmed, probable, suspected...           69   
 

0
Country Date    Variable                                                
Liberia 2014-06 Total death/s in confirmed, probable, suspected...    98
        2014-07 Total death/s in confirmed, probable, suspected...   290
        2014-08 Total death/s in confirmed, probable, suspected...  1770
        2014-09 Total death/s in confirmed, probable, suspected...  4116
        2014-10 Total death/s in confirmed, probable, suspected...  5474
        2014-11 Total death/s in confirmed, probable, suspected...  5400

0
Country Date    Variable                                                
Liberia 2014-06 Total death/s in confirmed, probable, suspected...    98
        2014-07 Total death/s in confirmed, probable, suspected...   192
        2014-08 Total death/s in confirmed, probable, suspected...  1480
        2014-09 Total death/s in confirmed, probable, suspected...  2346
        2014-10 Total death/s in confirmed, probable, suspected...  1358
        2014-11 Total death/s in confirmed, probable, suspected...   -74

In [154]:
data.groupby(data.Country).agg(np.mean).head()


,34 Military Hospital,Bo EMC,Hastings-F/Town,Kenema (IFRC),Police training School,Police traning School,Unnamed: 18,Western area,Western area combined,Bomi County,...,Grand Gedeh,Grand Kru,Lofa County,Margibi County,Maryland County,Montserrado County,Nimba County,River Gee County,RiverCess County,Sinoe County
Country,,,,,,,,,,,,,,,,,,,,,
Guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Liberia,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,NaN,39.652187,...,1.421163,5.753939,90.534113,194.645789,8.967164,512.878995,48.895433,2.087405,5.599309,5.522935
Sierra-Leone,19.285714,9.857143,81.696429,24.540146,7.428571,NaN,NaN,87709.792617,1304507.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
l.head()

,Bomi County,Bong County,Date,Gbarpolu County,Grand Bassa,Grand Cape Mount,Grand Gedeh,Grand Kru,Lofa County,Margibi County,Maryland County,Montserrado County,National,Nimba County,River Gee County,RiverCess County,Sinoe County,Unnamed: 18,Variable,Country
0,NaN,NaN,2014-06-16,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,Specimens collected,Liberia
1,NaN,NaN,2014-06-16,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Specimens pending for testing,Liberia
2,NaN,NaN,2014-06-16,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,7.0,28.0,NaN,NaN,NaN,NaN,NaN,Total specimens tested,Liberia
3,NaN,NaN,2014-06-16,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,2.0,NaN,NaN,NaN,NaN,NaN,Newly reported deaths,Liberia
4,NaN,NaN,2014-06-16,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,0.0,8.0,NaN,NaN,NaN,NaN,NaN,Total death/s in confirmed cases,Liberia


In [70]:
# Write your answer here
ebola_path = DATA_FOLDER + '/ebola'
country = ['/guinea_data', '/liberia_data', '/sl_data']
csv_file_list = []#csv_file_list+glob.glob(ebola_path+country[0]+'/*.csv')
for c in country:
    path = ebola_path+c+'/*.csv'
    #print(path)
    csv_file_list=csv_file_list+glob.glob(path)
#csv_file_list = glob.glob(ebola_path+country[0]+'/*.csv')
#print(csv_file_list)
guinea = pd.read_csv(csv_file_list[0])
for f in csv_file_list[1:]:
    temp = pd.read_csv(f)
    pd.merge(guinea,temp, how = 'outer')
guinea





MergeError: No common columns to perform merge on

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here